## Introdução
Nessa atividade iremos implementar várias instanciações do modelo vetorial e comparar suas eficiências em termos de Mean Average Precision (MAP). Os dados são sobre notícias e estão organizados em um arquivo csv que possui as seguintes colunas:

<i> timestamp, titulo, subTitulo, conteúdo, url, idNoticia </i>

Os nomes das colunas são bastante claros e autodescritivos.

Na tabela abaixo podemos ver uma pequena porção dos dados.


In [1]:
import pandas as pd

news_df = pd.read_csv("data/estadao_noticias_eleicao.csv", engine = "python")

news_df.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,NaN,Veja as principais fotos do dia e dos eventos ...,NaN,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,NaN,Veja as principais fotos do dia e dos eventos ...,NaN,"http://fotos.estadao.com.br/fotos/politica,dil...",5


Vamos realizar a tarefa em pequenos passos.

### Passo 1: Construir um índice básico considerando o conjunto de notícias;

Iremos construir um índice invertido simples (igual ao da tarefa anterior). Mas antes de construir esse índece irei substituir alguns valores faltantes (NaN - Not a Number) por strings vazias para que não haja problemas ao manipularmos essas colunas do dataframe.

In [2]:
# Pre-processamento
news_df.fillna(" ", inplace = True)

news_df.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


Como podemos ver, os valores faltantes (NaN) foram substituídos por vazio.

In [3]:
import nltk
from nltk.tokenize import RegexpTokenizer

"""Cria uma estrutura de índice invertido.

argumentos:
df -- Um dataframe pandas 
"""
def create_indexer(df):
    global indexer
    indexer = {}

    for index, row in df.iterrows():
        document = row['titulo'] + " " + row['subTitulo'] + " " + row['conteudo']
        document_words = preprocess(document)
        doc_id = row['idNoticia']

        for term in document_words:
            if term in indexer:
                indexer[term].add(doc_id)
            else:
                indexer[term] = set([doc_id])

"""Transforma uma string, convertendo-a para caixa baixa e removendo a pontuação."""
def preprocess(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = text.lower()
    return tokenizer.tokenize(text)

# Cria o índece invertido, armazenando-o na variável global 'indexer'
create_indexer(news_df)

### Passo 2: Refinamento do índice

Nesta etapa iremos melhorar o índice invertido de forma a incluir o IDF (inverse document frequency) de cada termo do dicionário e o TF (term frequency) de cada termo em cada documento da lista de postings respectiva.

TF é a abreviação para <i>frequência do termo</i>. Ele marca o quão frequente determinado termo é no documento.

IDF é a abreviação para <i>frequência de documento invertida</i>. O IDF é dado pela seguinte fórmula:

IDF(termo) = log[(M+1)/k], onde M é o número total de documentos na coleção e K é o número total de documentos que contêm o termo.

Abaixo encontra-se o código da função que cria o índice refinado, onde cada doc_id é a chave para uma frequência de termo.

O IDF será calculado em seguida, pois é só aplicar as fórmulas nas informações que já temos, que são o número de documentos na coleção e o número de documentos que contêm cada termo.

In [4]:
"""Cria uma estrutura de índice invertido com as informações de 
Frequência de Termo (TF) e Frequência de Documento Invertida (IDF).

argumentos:
df -- Um dataframe pandas 
"""
def create_indexer(df):
    global indexer
    indexer = {}

    for index, row in df.iterrows():
        document = row['titulo'] + " " + row['subTitulo'] + " " + row['conteudo']
        document_words = preprocess(document)
        doc_id = row['idNoticia']

        for term in document_words:
            if term in indexer:
                posting = indexer[term]
                if (doc_id in posting):
                    TF = indexer[term][doc_id] + 1
                    indexer[term][doc_id] = TF
                else:
                    indexer[term][doc_id] = 1
            else:
                indexer[term] = {doc_id: 1}
                
    indexer["__all_docs_count__"] = df.shape[0]
                
                
# Cria o índece invertido, armazenando-o na variável global 'indexer'
create_indexer(news_df)


### Instanciar o modelo vetorial

Nesta etapa da tarefa iremos implementar 4 versões do modelo vetorial. São elas:

1. representação binária;
2. TF;
3. TF-IDF;
4. BM25* (não usaremos Okapi já que os documentos não tem variação de tamanho).

O modelo vetorial é diferente do modelo de busca binária sumariamente porque consegue ranquear os documentos por ordem de relevância. Cada técnica acima utiliza uma estratégia diferente para definir esta relevância, estando apresentadas em ordem de menos complexa para mais complexa.

Abaixo encontra-se as funções que implementam estas estratégias.

In [11]:
import math

# HELPERS:
"""
Conta a frequência das palavras em uma lista de palavras.

argumentos:
list_of_words -- A lista de palavras.
"""
def word_freq(list_of_words):
    result = {}
    
    for word in list_of_words:  # Conta a frequência das palavras na consulta
        if word in result:
            result[word] = result[word] + 1 
        else:
            result[word] = 1
    
    return result

"""
Calcula o IDF (Inverse Document Frequency) de um termo no índice.

argumentos:
total_docs -- O número total de documentos no índice.
doc_frequency -- A quantidade de documentos que contêm o termo.
"""
def IDF(total_docs, doc_frequency):
    return math.log((total_docs + 1) / doc_frequency)

"""
Calcula o BM25 de um termo em um documento no índice.

argumentos:
term_freq -- A quantidade de vezes que o termo aparece no documento.
k - O parâmetro de ajuste do algorítmo.
"""
def BM25(term_freq, k = 5): 
    return (((k + 1) * term_freq) / (term_freq + k))


# ALGORÍTMOS:

def binary_VSM(query):
    query_words = preprocess(query)
    
    docs = {} # dicionário que guarda os resultados <doc_id: pontuação-no-ranking>
    
    for term in set(query_words): # Calcula o ranking para cada documento
        posting = indexer[term]   # documentos com aquela palavra
        for doc_id in posting:
            if doc_id in docs:
                docs[doc_id] = docs[doc_id] + 1 
            else:
                docs[doc_id] = 1
                
    ranked_docs = sorted(docs, key = docs.get, reverse = True)
    
    return ranked_docs


def TF_VSM(query):
    query_words = preprocess(query)
    
    query_terms = word_freq(query_words)
    
    docs = {}  # dicionário que guarda os resultados <doc_id: pontuação-no-ranking>
    
    for term in query_terms:  # Calcula o ranking para cada documento utilizando TF
        query_freq = query_terms[term]  # a frequência da palavra na consulta
        posting = indexer[term]         # documentos com aquela palavra
        for doc_id in posting:
            score_for_term = posting[doc_id] * query_freq # Calcula a pontuação do documento para certo termo
            if doc_id in docs:             
                docs[doc_id] = docs[doc_id] + score_for_term 
            else:
                docs[doc_id] = score_for_term
                
    ranked_docs = sorted(docs, key = docs.get, reverse = True)
    
    return ranked_docs
             
def TFIDF_VSM(query):
    query_words = preprocess(query)
    
    query_terms = word_freq(query_words)
    
    docs = {}  # dicionário que guarda os resultados <doc_id: pontuação-no-ranking>
    
    total_docs = indexer["__all_docs_count__"]
    
    for term in query_terms:  # Calcula o ranking para cada documento utilizando TF
        query_freq = query_terms[term]   # a frequência da palavra na consulta
        posting = indexer[term]          # documentos com aquela palavra
        docs_freq = len(posting)         # a frequencia da palavra em documentos
        idf = IDF(total_docs, docs_freq)
        for doc_id in posting:
            score_for_term = posting[doc_id] * query_freq * idf # Calcula a pontuação do documento para certo termo
            if doc_id in docs:             
                docs[doc_id] = docs[doc_id] + score_for_term 
            else:
                docs[doc_id] = score_for_term
                
    ranked_docs = sorted(docs, key = docs.get, reverse = True)
    
    return ranked_docs


def BM25_VSM(query):
    query_words = preprocess(query)
    
    query_terms = word_freq(query_words)
    
    docs = {}  # dicionário que guarda os resultados <doc_id: pontuação-no-ranking>
    
    total_docs = indexer["__all_docs_count__"]
    
    for term in query_terms:  # Calcula o ranking para cada documento utilizando TF
        query_freq = query_terms[term]   # a frequência da palavra na consulta
        posting = indexer[term]          # documentos com aquele termo
        docs_freq = len(posting)         # a frequencia do termo em todos os documentos
        idf = IDF(total_docs, docs_freq)
        for doc_id in posting:
            term_freq = posting[doc_id]  # frequencia do termo no documento
            bm25 = BM25(term_freq)
            score_for_term = query_freq * bm25 * idf # Calcula a pontuação do documento para certo termo
            if doc_id in docs:             
                docs[doc_id] = docs[doc_id] + score_for_term 
            else:
                docs[doc_id] = score_for_term
                
    ranked_docs = sorted(docs, key = docs.get, reverse = True)
    
    return ranked_docs

### Passo 4: Executar os algoritmos

Agora vamos executar os algorítmos nas consultas abaixo e ver os resultados obtidos para os top 5 documentos em cada caso.

1. segundo turno;
2. lava jato;
3. projeto de lei;
4. compra de voto.
5. ministério público.

In [29]:
queries = ["segundo turno", "lava jato", "projeto de lei",
           "compra de voto", "ministério público"]

results = pd.DataFrame(columns = ["consulta", "binary", "tf", "tfidf", "bm25"])

index = 0
for q in queries:
    results.loc[index] = [q, 
                          str(binary_VSM(q)[:5]),
                          str(TF_VSM(q)[:5]),
                          str(TFIDF_VSM(q)[:5]),
                          str(BM25_VSM(q)[:5])]
    index = index + 1
    
results.head()

,consulta,binary,tf,tfidf,bm25
0,segundo turno,"[1, 7, 13, 26, 69]","[2744, 7, 2112, 7672, 2388]","[2744, 2112, 7672, 1235, 2388]","[2744, 2112, 7672, 2388, 2178]"
1,lava jato,"[3, 13, 15, 27, 43]","[163, 353, 6942, 2807, 127]","[163, 353, 6942, 2807, 127]","[163, 353, 6942, 2807, 127]"
2,projeto de lei,"[7, 10, 25, 38, 56]","[7, 155, 6554, 3942, 7017]","[7017, 5129, 6096, 2853, 155]","[2853, 2232, 3171, 6699, 6461]"
3,compra de voto,"[82, 347, 553, 748, 854]","[7, 155, 6554, 3942, 7017]","[173, 2047, 7017, 7343, 7293]","[7343, 2047, 2200, 7293, 173]"
4,ministério público,"[7, 15, 21, 27, 38]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]"


### Passo 5: Avaliar os algoritmos


Para avaliarmos os nossos diferentes rankings iremos utilizar uma métrica bastante conhecida, a MAP (Mean Average Precision), que calcula a precisão média em cada corte do ranking em que um novo documento relevante é recuperado.

Abaixo temos o código de uma função que implementa a MAP retirada [deste link](https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py).
Iremos utilizar esta função para avaliarmos os modelos juntamente com um gabarito que foi fornecido.

In [13]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

Abaixo temos um visão do gabarito, que contem o ranking formulado pelo google e por outras fontes seguindo esses mesmos algoritmos e em seguida o resultado das comparações.

In [8]:
answers = pd.read_csv("data/gabarito.csv", engine = "python")

answers.head()

,str_busca,google,busca_binaria,tf,tfidf,bm25
0,segundo turno,"[1062, 1942, 2161, 2078, 2073]","[2048, 1, 2049, 2050, 4096]","[2744, 7, 2112, 7672, 2388]","[2744, 2112, 7672, 1235, 2388]","[2744, 2112, 7672, 2388, 2178]"
1,lava jato,"[616, 164, 1734, 163, 6716]","[3, 13, 15, 27, 6177]","[163, 353, 2807, 127, 359]","[163, 353, 2807, 127, 359]","[163, 353, 2807, 127, 359]"
2,projeto de lei,"[2853, 275, 978, 7092, 3171]","[3584, 6145, 8194, 8706, 6660]","[7, 3942, 7017, 1250, 6942]","[2232, 6461, 2853, 3171, 3942]","[2232, 6461, 3171, 2853, 3170]"
3,compra de voto,"[2200, 8615, 2265, 7746, 82]","[7424, 2178, 6531, 5122, 2311]","[3942, 7017, 5129, 2047, 748]","[7343, 7293, 6791, 3942, 2047]","[7343, 7293, 6791, 7329, 8615]"
4,ministério público,"[64, 6652, 164, 6550, 8615]","[8194, 7, 4104, 8201, 4109]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]"


In [39]:
import re

"""
Converte uma lista de listas stringfadas em sua versão de lista de listas

argumento:
list_of_lists -- a lista de strings, onde cada string é uma lista stringfada.
"""
def parse_list_int(list_of_lists):
    result = []
    
    for lis in list_of_lists:        
        as_str = re.sub('[,\[\]]', '', lis)
        as_list = as_str.split(" ")
        list_of_int = list(map(int, as_list))
        result.append(list_of_int)
    
    return result

print("Comparando com o gabarito fornecido:")
print("Busca binária: " + str(mapk(parse_list_int(answers.busca_binaria), parse_list_int(results.binary), k=5)))
print("           TF: " + str(mapk(parse_list_int(answers.tf), parse_list_int(results.tf), k=5)))
print("       TF-IDF: " + str(mapk(parse_list_int(answers.tfidf), parse_list_int(results.tfidf), k=5)))
print("         BM25: " + str(mapk(parse_list_int(answers.bm25), parse_list_int(results.bm25), k=5)))
print("\n")
print("Comparando com o google:")
print("Busca binária: " + str(mapk(parse_list_int(answers.google), parse_list_int(results.binary), k=5)))
print("           TF: " + str(mapk(parse_list_int(answers.google), parse_list_int(results.tf), k=5)))
print("       TF-IDF: " + str(mapk(parse_list_int(answers.google), parse_list_int(results.tfidf), k=5)))
print("         BM25: " + str(mapk(parse_list_int(answers.google), parse_list_int(results.bm25), k=5)))

Comparando com o gabarito fornecido:
Busca binária: 0.24
           TF: 0.6519999999999999
       TF-IDF: 0.616
         BM25: 0.7539999999999999


Comparando com o google:
Busca binária: 0.04
           TF: 0.048
       TF-IDF: 0.057999999999999996
         BM25: 0.128


### Análise dos resultados
##### Do melhor ao pior, como se saíram os modelos?

Na comparação com o gabarito fornecido, os melhores modelos foram em ordem de maior MAP:

1. BM25 (0.754)
2. TF (0.652)
3. TF-IDF (0.616)
4. Busca binária (0.24)

Na comparação com o google, os resultados foram:

1. BM25 (0.128)
2. TF-IDF (0.058)
3. TF (0.048)
4. Busca binária (0.04)

Os resultados estão na linha do que se era esperado. Quanto mais próximo de 1, melhor o MAP. Com um MAP de 0,5, por exemplo, sabemos que a cada 10 documentos recuperados, 5 estarão corretos.

Na comparação com o google nenhum dos modelos apresentou um bom MAP, sendo o maior igual a 0.128, que foi o do modelo BM25. Acredito que o resultado tenha sido assim pois o google é uma ferramenta de busca muito mais complexa e potente do que estes simples modelos.

Na comparação com o gabarito fornecido os resultados foram bons na maioria dos casos. Em 3 dos 4 modelos o MAP ficou acima de 0,6 e somente no modelo mais básico, o da busca binária, o MAP apresentou um valor baixo, 0,24. 

Em ambas as comparações, o BM25 apresentou o melhor resultado e a busca binária o pior resultado. Através desse simples experimento podemos perceber que quanto mais complexo o modelo, melhor a sua precisão. 